In [1]:
import pandas as pd
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
import sklearn.model_selection as model_selection
import tensorflow_hub as hub
import tensorflow as tf
import re
import random
import html

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from urllib.parse import urlparse, urlsplit
from math import floor, ceil
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc, classification_report, balanced_accuracy_score
from sklearn.utils import shuffle
from scipy import stats
from scipy.stats import spearmanr, rankdata, mode
from transformers.modeling_tf_distilbert import TFDistilBertModel
from transformers.modeling_tf_roberta import TFRobertaModel
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForTokenClassification,
    BertTokenizer,
    BertForPreTraining,
    BertModel,
    CamembertConfig,
    CamembertForTokenClassification,
    CamembertTokenizer,
    DistilBertModel,
    DistilBertConfig,
    DistilBertForTokenClassification,
    DistilBertTokenizer,
    RobertaConfig,
    RobertaModel,
    RobertaForMaskedLM,
    RobertaForTokenClassification,
    RobertaTokenizer,
    XLMRobertaConfig,
    XLMRobertaForTokenClassification,
    XLMRobertaTokenizer,
    get_linear_schedule_with_warmup,
)

/home/pauldudko/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pauldudko/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pauldudko/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pauldudko/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
!ls

data  tdr-ppcs.ipynb  tdr.ipynb


In [3]:
troll_df = pd.read_csv("data/comments.csv", error_bad_lines=False)
normie_df = pd.read_csv("data/csv-zusammenfuehren.de_3mjg6fs7.csv", error_bad_lines=False)

b'Skipping line 4581: expected 20 fields, saw 22\nSkipping line 9684: expected 20 fields, saw 21\nSkipping line 11215: expected 20 fields, saw 27\n'
b'Skipping line 33342: expected 20 fields, saw 21\nSkipping line 38620: expected 20 fields, saw 22\nSkipping line 40411: expected 20 fields, saw 21\n'
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
troll_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6711 entries, 0 to 6710
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   fullname                         6711 non-null   object 
 1   archived                         6711 non-null   bool   
 2   author.name                      6711 non-null   object 
 3   author_flair_text                807 non-null    object 
 4   body                             6711 non-null   object 
 5   controversiality                 6711 non-null   int64  
 6   created_utc                      6711 non-null   float64
 7   distinguished                    0 non-null      float64
 8   downs                            6711 non-null   int64  
 9   edited                           6711 non-null   object 
 10  gilded                           6711 non-null   int64  
 11  id                               6711 non-null   object 
 12  is_submitter        

In [5]:
troll_df['controversiality'].describe()

count    6711.000000
mean        0.036507
std         0.187563
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: controversiality, dtype: float64

In [6]:
normie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45143 entries, 0 to 45142
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   body                    45143 non-null  object 
 1   score_hidden            11985 non-null  object 
 2   archived                11982 non-null  object 
 3   name                    11981 non-null  object 
 4   author                  45131 non-null  object 
 5   author_flair_text       11461 non-null  object 
 6   downs                   11979 non-null  object 
 7   created_utc             45129 non-null  float64
 8   subreddit_id            45129 non-null  object 
 9   link_id                 45129 non-null  object 
 10  parent_id               45129 non-null  object 
 11  score                   45129 non-null  float64
 12  retrieved_on            44990 non-null  float64
 13  controversiality        45129 non-null  float64
 14  gilded                  45129 non-null

In [7]:
normie_df['controversiality'].describe()

count    45129.000000
mean         0.020585
std          0.141993
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: controversiality, dtype: float64

In [8]:
troll_df.head()

,fullname,archived,author.name,author_flair_text,body,controversiality,created_utc,distinguished,downs,edited,...,link_url,name,num_comments,parent_id,permalink,score,stickied,subreddit.display_name_prefixed,subreddit_type,ups
0,t1_d687zh5,True,BlackToLive,NaN,"A hard look at training and tactics"" = They wi...",0,1.470604e+09,NaN,0,False,...,http://www.reuters.com/article/us-usa-police-c...,t1_d687zh5,119,t3_4wkn7m,/r/Bad_Cop_No_Donut/comments/4wkn7m/chicago_po...,1,False,r/Bad_Cop_No_Donut,public,1
1,t1_d5wqzhx,True,BlackToLive,NaN,They deserve all of the hate,0,1.469847e+09,NaN,0,False,...,http://reason.com/blog/2016/07/28/pine-bluff-c...,t1_d5wqzhx,96,t3_4v5xpc,/r/Bad_Cop_No_Donut/comments/4v5xpc/arkansas_p...,1,False,r/Bad_Cop_No_Donut,public,1
2,t1_d5qvqfw,True,BlackToLive,NaN,"I guess that's what they mean when say ""I don'...",0,1.469498e+09,NaN,0,False,...,http://countercurrentnews.com/2016/07/no-charg...,t1_d5qvqfw,210,t1_d5qeyrw,/r/Bad_Cop_No_Donut/comments/4uiezg/no_charges...,1,False,r/Bad_Cop_No_Donut,public,1
3,t1_d5quz9y,True,BlackToLive,NaN,"It's never too late for them, It's never too c...",0,1.469497e+09,NaN,0,False,...,http://www.opposingviews.com/i/society/police-...,t1_d5quz9y,18,t3_4uicjv,/r/Bad_Cop_No_Donut/comments/4uicjv/police_off...,1,False,r/Bad_Cop_No_Donut,public,1
4,t1_d565ls1,True,BlackToLive,NaN,https://petitions.whitehouse.gov//petition/pet...,0,1.468114e+09,NaN,0,False,...,http://www.thelibertyconservative.com/favor-po...,t1_d565ls1,12,t1_d55o1gr,/r/Good_Cop_Free_Donut/comments/4s0s3j/you_can...,1,False,r/Good_Cop_Free_Donut,public,1


In [9]:
troll_df['subreddit'] = troll_df["subreddit.display_name_prefixed"].apply(lambda x: x.split("/")[1])

In [10]:
troll_df.head().T

,0,1,2,3,4
fullname,t1_d687zh5,t1_d5wqzhx,t1_d5qvqfw,t1_d5quz9y,t1_d565ls1
archived,True,True,True,True,True
author.name,BlackToLive,BlackToLive,BlackToLive,BlackToLive,BlackToLive
author_flair_text,NaN,NaN,NaN,NaN,NaN
body,"A hard look at training and tactics"" = They wi...",They deserve all of the hate,"I guess that's what they mean when say ""I don'...","It's never too late for them, It's never too c...",https://petitions.whitehouse.gov//petition/pet...
controversiality,0,0,0,0,0
created_utc,1.4706e+09,1.46985e+09,1.4695e+09,1.4695e+09,1.46811e+09
distinguished,NaN,NaN,NaN,NaN,NaN
downs,0,0,0,0,0
edited,False,False,False,False,False


In [11]:
normie_df.head().T

,0,1,2,3,4
body,"I guess there was some awkwardness, but it was...","Food schmooze isn't bad, I've gotten a few int...",Is there a fatigue animation before sleep that...,"I recently saw some Shohei Inamura films, spec...",How long did that take?
score_hidden,false,false,false,false,false
archived,false,false,false,false,false
name,t1_cob0s2g,t1_cov0mx1,t1_colfupi,t1_coyhhvn,t1_cos5df5
author,elevenghosts,BeerBaldBeard,tmrxwoot,Zassolluto711,Ditto_B
author_flair_text,NaN,NaN,NaN,NaN,NaN
downs,0,0,0,0,0
created_utc,1.42307e+09,1.42474e+09,1.42394e+09,1.425e+09,1.42449e+09
subreddit_id,t5_2qh1i,t5_2qji0,t5_2rron,t5_2qh3s,t5_2qtol
link_id,t3_2urgi1,t3_2wj4w8,t3_2vvy1m,t3_2x9q0x,t3_2w566b


In [12]:
def url_tracker(text):
    find = re.compile('www\.[\S]+|https?://[\S]+[\w|/]')
    urls = re.findall(find, html.unescape(text))
    uri = ""
    if len(urls)>0:

        for url in urls:
            if url != "":
                uri = uri+" "+netloc_parser(url)+" "+url_parser(url)+" "
    return uri

def url_parser(text):
    path = urlsplit(text).path
    words = re.findall(r"[\w']+", path)
    uri = ""
    for word in words: 
        uri = uri + word + " "
    uri.strip()
    return uri

def netloc_parser(text):
    ntlc = urlsplit(text).netloc.split(".")
    uri = ntlc[0]
    
    if uri == "www":
        uri = ntlc[1]
        
    if uri == "reddit":
        uri = ""
    elif uri == "i":
        uri = "imgur"
    elif uri == "youtu":
        uri = "youtube"
    
    return uri

normie_df['link_data'] = normie_df['body'].apply(lambda x : url_tracker(x))

In [13]:
SYMBOLS_TO_ISOLATE = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
SYMBOLS_TO_REMOVE = '\n\r\xa0\ue014\t\uf818\uf04a\xad\uf0e0\u200b\u200eعدويهصقأناخلىبمغرЕ\u202a\u202c🏻ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢשלוםבי‼\x81エンジ故障\u2009ᴵ͞\u200fאעכחஜᴠ‐\x7fἐὶήιὲκἀίῃἴξＨ\ufeff\u2028\u3000تحكسة👮💙فزط\u2008🏾\x08‑地獄谷улкнПоАН歌舞伎הмυтѕ⤵\u200aэпрд\x95\u2002\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13\ue602άοόςέὸתמדףנרךצט\uf0b7\uf04c\x9f\x10成都\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス外国人关系Сб💋💀🎄💜ьыгя不是\x9c\x9d🗑\u2005💃📣༼つ༽ḷЗз▱ц￼卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡й\u2003🚀🤴ʲшчИОРФДЯМюж🖑ὐύύ特殊作戦群щ💨圆明园קℐ\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'
POSITIVE_EMOJI = '😜😎😁💖😀😂😄😋👏😊👍😃😘👌🙂😉😏🎉😅👻🙃😆🤗🤓😌🤑😛🤣😝💪😗🥰😇🤠🤡🥳🥴🤩😺😸😹😻😽✌️🤟🤘'
NEGATIVE_EMOJI = '😢👎😱😳😧🙀😐😕😮😖😟😡😠😤😞😭😥😔😓😪😨😩🙁😵😒͝😣😲😯🤢َِ😰👿👿🤥😬😷🤒🤕🤯🤬🥺🙀😿😾🖕🏻🖕🏼'
NEUTRAL_EMOJI = '🐶️🍕🐵💵🔥💥🚌🌟💩💯⛽🚄🏼🚲😈🙏🎯🌹💔👊🙄⛺🍾🏽🎆🍻⏺🌏💞🚓🔔📚🏀👐🍽🎶🌺🤔🐰🐇🏈😺🌍🍔🐮🍁🍆🍑🌮🌯🤦🙈😴🆕👅👥👄🔄🔤👉👤🤧👶👲🔛🎓🏿🇺🇸🌠🐟💫💰🚬💎🐱🙆💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚🐾🐕🔗🚽🏆🎃🖐🙅⛲🍰🤐👆🙌💛👀🙊🙉'
ISOLATE_DICT = {ord(c):' special symbol '.format(ord(c)) for c in SYMBOLS_TO_ISOLATE}
REMOVE_DICT = {ord(c):'' for c in SYMBOLS_TO_REMOVE}
NEUTRAL_EMOJI_DICT = {ord(c):' neutral emoji ' for c in NEUTRAL_EMOJI}
POSITIVE_EMOJI_DICT = {ord(c):' positive emoji ' for c in POSITIVE_EMOJI}
NEGATIVE_EMOJI_DICT = {ord(c):' negative emoji ' for c in NEGATIVE_EMOJI}
CONTRACTION_MAPPING = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }


def handle_punctuation(text):
    text = text.translate(REMOVE_DICT)
    text = text.translate(NEUTRAL_EMOJI_DICT)
    text = text.translate(POSITIVE_EMOJI_DICT)
    text = text.translate(NEGATIVE_EMOJI_DICT)
    text = text.translate(ISOLATE_DICT)
    return text

def clean_contractions(text, mapping=CONTRACTION_MAPPING):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text


def text_general_reworking(text):
    spaces = ['\u200b', '\u200e', '\u202a', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\x10', '\x7f', '\x9d', '\xad',
              '\xa0']
    for space in spaces:
        text = text.replace(space, ' ')
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\.{2,}', ' ', text)
    text = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' positive emoji ', text)
    text = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' positive emoji ', text)
    text = re.sub(r'(<3|:\*)', ' positive emoji ', text)
    text = re.sub(r'(;-?\)|;-?D|\(-?;)', ' positive emoji ', text)
    text = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' negative emoji ', text)
    text = re.sub(r'(:,\(|:\'\(|:"\()', ' negative emoji ', text)
    text = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' link ', text)
    text = re.sub(r'\brt\b', '', text)
    text = re.sub(r'\[math\]', ' LaTex math ', text)
    text = re.sub(r'\[\/math\]', ' LaTex math ', text)
    text = re.sub(r'\\', ' LaTex ', text)
    text = re.sub(r'\brt\b', '', text)
    text = re.sub("([^\"^\'].\s)(\")([A-Z,a-z?])", r"\1\3", text)
    text = re.sub("(\')(.\")", r"\2", text)
    text = text.lower()
    if '-' in text:
        text = text.replace('-', ' - ')
    text.strip()
    return text

def text_preprocess(x):
    x = text_general_reworking(x)
    x = handle_punctuation(x)
    x = clean_contractions(x)
    return x


troll_df['netloc'] = troll_df['link_url'].apply(lambda x : url_parser(x))
troll_df['link_data'] = troll_df['link_title'].astype('str') + " " + troll_df['netloc'].astype('str')
#normie_df['body'] = normie_df['link_data'].astype('str') + " " + normie_df['body'].astype('str')

troll_df['body'] = troll_df['body'].apply(html.unescape)
normie_df['body'] = normie_df['body'].apply(html.unescape)
troll_df['body'] = troll_df['body'].apply(lambda x: text_preprocess(x))
troll_df['link_data'] = troll_df['link_data'].apply(lambda x: text_preprocess(x))
normie_df['body'] = normie_df['body'].apply(lambda x: text_preprocess(x))
normie_df['link_data'] = normie_df['link_data'].apply(lambda x: text_preprocess(x))

In [14]:
troll_df.dropna(subset = ['body','subreddit'], inplace = True)
normie_df.dropna(subset = ['body','subreddit'], inplace = True)
normie_df = normie_df[(normie_df['body'].astype('str')!='[deleted]')  &  (normie_df['body'].astype('str')!='[removed]')]
normie_df.drop_duplicates(inplace=True)

In [15]:
troll_df['troll'] = True
normie_df['troll'] = False

In [16]:
cols = ["body",'created_utc','link_data',"subreddit","score",'controversiality', "troll"]

In [17]:
normie_link_df = normie_df[(normie_df['link_data'].astype('str')!="") & (normie_df['controversiality']==0)][cols].copy()

In [18]:
data_df = troll_df[cols]
normie_contro_df = normie_df[normie_df['controversiality']==1][cols].copy()
data_df = data_df.append(normie_contro_df)
n = troll_df.shape[0] - normie_contro_df.shape[0] 
data_df = data_df.append(normie_df[normie_df['controversiality']==0][cols].sample(n))
data_df = shuffle(data_df)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13422 entries, 6174 to 8174
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   body              13422 non-null  object 
 1   created_utc       13422 non-null  float64
 2   link_data         13422 non-null  object 
 3   subreddit         13422 non-null  object 
 4   score             13422 non-null  float64
 5   controversiality  13422 non-null  float64
 6   troll             13422 non-null  bool   
dtypes: bool(1), float64(3), object(3)
memory usage: 747.1+ KB


In [19]:
data_df['body'] = data_df['body'].astype('str')
data_df['link_data'] = data_df['link_data'].astype('str')
data_df['subreddit'] = data_df['subreddit'].astype('str')
data_df['troll'] = data_df['troll'].astype('int')
data_df['controversiality'] = data_df['controversiality'].astype('int')

def subreddit_decoder(text):
    if '_' in text:
        text = text.replace('_', ' ')
    text = re.sub("([a-z])([A-Z])", r"\1 \2", text)
    return text

data_df['subreddit'] = data_df['subreddit'].apply(lambda x: subreddit_decoder(x))

In [20]:
data_df.head(10)

,body,created_utc,link_data,subreddit,score,controversiality,troll
6174,special symbol france threatens haha,1.443650e+09,france threatens to walk away from ttip negoti...,worldnews,1.0,0,1
2610,future is here special symbol,1.515363e+09,ties special symbol network is the platform fo...,Database,1.0,0,1
38188,special symbol removed special symbol,1.493285e+09,,Official DP,-3.0,0,0
5908,we do special symbol i did not know that spec...,1.429122e+09,,pics,3.0,1,0
1711,run special symbol forrest special symbol ru...,1.433680e+09,my sweet cat zagcmdl jpg,pics,2.0,0,1
29117,eh special symbol there are new 0days all the...,1.478784e+09,,techsupport,1.0,0,0
3336,haha special symbol seems that mr special sym...,1.446211e+09,cops across the nation join nypd in tarantino ...,Bad Cop No Donut,1.0,0,1
44552,special symbol you are assuming on faith tha...,1.484082e+09,,Capitalism VSocialism,1.0,0,0
2437,have we actually seen anything from ibm regard...,1.515451e+09,ibm executive brief special symbol three ways...,Stellar,1.0,0,1
28397,good luck special symbol lol special symbol ...,1.475424e+09,,Silverbugs,1.0,0,0


In [21]:
pd.crosstab(normie_df['controversiality'], normie_df['troll'])

troll,False
controversiality,
0.0,44200
1.0,929


In [22]:
pd.crosstab(troll_df['controversiality'], troll_df['troll'])

troll,True
controversiality,
0,6466
1,245


In [23]:
pd.crosstab(data_df['controversiality'], data_df['troll'], margins = True)

troll,0,1,All
controversiality,,,
0,5782,6466,12248
1,929,245,1174
All,6711,6711,13422


In [24]:
subr_len = data_df['subreddit'].map(str).apply(len).describe()
body_len = data_df['body'].map(str).apply(len).describe()
link_len = data_df['link_data'].map(str).apply(len).describe()
print(subr_len, body_len, link_len)

count    13422.000000
mean         9.748473
std          4.417478
min          2.000000
25%          6.000000
50%         10.000000
75%         13.000000
max         25.000000
Name: subreddit, dtype: float64 count    13422.000000
mean       192.142453
std        377.289469
min          1.000000
25%         44.000000
50%         98.000000
75%        211.000000
max      14164.000000
Name: body, dtype: float64 count    13422.000000
mean        88.395545
std        116.963048
min          0.000000
25%          0.000000
50%         33.000000
75%        157.000000
max       4216.000000
Name: link_data, dtype: float64


In [25]:
MAX_SEQUENCE_LENGTH = 512
QUESTION_MAX_LENGTH = 483
ANSWER_MAX_LENGTH = 25

### Code below is partly based on:

https://www.kaggle.com/shuheigoda/23th-place-solusion
https://github.com/oleg-yaroshevskiy/quest_qa_labeling/blob/master/model.py
https://www.kaggle.com/abhishek/distilbert-use-features-oof

In [26]:
def trim_input(question_tokens, answer_tokens, max_seq_len, q_max_len, a_max_len):
    q_len = len(question_tokens)
    a_len = len(answer_tokens)
    if q_len + a_len + 4 > max_seq_len:
        if a_max_len <= a_len and q_max_len <= q_len:
            q_new_len_head = floor((q_max_len - q_max_len/2))
            question_tokens = question_tokens[:q_new_len_head] + question_tokens[q_new_len_head - q_max_len:]
            a_new_len_head = floor((a_max_len - a_max_len/2))
            answer_tokens = answer_tokens[:a_new_len_head] + answer_tokens[a_new_len_head - a_max_len:]
        #elif q_len <= a_len and q_len < q_max_len:
        #   a_max_len = a_max_len + (q_max_len - q_len - 1)
        #   a_new_len_head = floor((a_max_len - a_max_len/2))
        #   answer_tokens = answer_tokens[:a_new_len_head] + answer_tokens[a_new_len_head - a_max_len:]
        elif a_len < q_len:
            assert a_len <= a_max_len
            q_max_len = q_max_len + (a_max_len - a_len - 1)
            q_new_len_head = floor((q_max_len - q_max_len/2))
            question_tokens = question_tokens[:q_new_len_head] + question_tokens[q_new_len_head - q_max_len:]
        else:
            raise ValueError("unreachable: q_len: {}, a_len: {}, q_max_len: {}, a_max_len: {}".format(q_len, a_len, q_max_len, a_max_len))
    return question_tokens, answer_tokens


def get_transformer_inputs(question, question_title, answer, tokenizer, question_only):
    question = "{} [SEP] {}".format(question_title, question)
    question_tokens = tokenizer.tokenize(question)
    if question_only:
        answer_tokens = []
    else:
        answer_tokens = tokenizer.tokenize(answer)
    question_tokens, answer_tokens = trim_input(question_tokens, answer_tokens, MAX_SEQUENCE_LENGTH, 
                                                QUESTION_MAX_LENGTH , ANSWER_MAX_LENGTH)
    ids = tokenizer.convert_tokens_to_ids(["[CLS]"] + question_tokens + ["[SEP]"] + answer_tokens + ["[SEP]"])
    padded_ids = ids + [tokenizer.pad_token_id] * (MAX_SEQUENCE_LENGTH - len(ids))
    token_type_ids = [0] * (1 + len(question_tokens) + 1) + [1] * (len(answer_tokens) + 1) \
                                                        + [0] * (MAX_SEQUENCE_LENGTH - len(ids))
    attention_mask = [1] * len(ids) + [0] * (MAX_SEQUENCE_LENGTH - len(ids))
    return padded_ids, token_type_ids, attention_mask

In [27]:
bert_tokenizer_path = '../models/bert/bert-base/bert-base-uncased-vocab.txt'
bert_tokenizer = BertTokenizer(vocab_file = bert_tokenizer_path)
sample_args = data_df["body"].values[15],data_df["link_data"].values[15], data_df["subreddit"].values[15]
sample_ids = get_transformer_inputs(*sample_args, bert_tokenizer, False)
print(sample_ids)
print(bert_tokenizer.convert_ids_to_tokens(sample_ids[0]))

([101, 102, 2065, 2017, 2064, 8054, 3087, 2008, 1996, 2280, 25467, 4005, 2085, 1999, 3094, 1997, 3607, 2003, 8699, 2017, 2323, 2131, 2019, 4469, 2833, 6463, 2078, 1998, 2028, 1997, 2216, 11281, 2845, 6665, 2569, 6454, 102, 3198, 8398, 6793, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [28]:
output_categories = ['troll','controversiality']

def compute_input_arrays(df, tokenizer, question_only=False):
    input_ids, input_token_type_ids, input_attention_masks = [], [], []
    for question, question_title, answer in zip(df["body"].values, df["link_data"].values, df["subreddit"].values):
        ids, type_ids, mask = get_transformer_inputs(question, question_title, answer, tokenizer, question_only=question_only)
        input_ids.append(ids)
        input_token_type_ids.append(type_ids)
        input_attention_masks.append(mask)
    return (
        np.asarray(input_ids, dtype=np.int32),
        np.asarray(input_token_type_ids, dtype=np.int32),
        np.asarray(input_attention_masks, dtype=np.int32)
    )

def compute_output_arrays(df):
    return np.asarray(df[output_categories], dtype=np.int32)

In [29]:
class BertStyleModel(torch.nn.Module):
    
    def __init__(self, model_type):
        super().__init__()
        
        self.model_type = model_type
        if (model_type == 'roberta'):
            config_path = '../models/bert/roberta-transformers-pytorch/roberta-base/roberta-base-config.json'
            model_path = '../models/bert/roberta-transformers-pytorch/roberta-base/roberta-base-pytorch_model.bin'
            config = RobertaConfig.from_json_file(config_path)
            config.output_hidden_states = True
            self.bert = RobertaModel.from_pretrained(model_path, config=config)
        elif (model_type == 'distilbert'):
            config_path = '../models/bert/distilbert-transformers-pytorch/distilbert-base-uncased-config.json'
            config = DistilBertConfig.from_json_file(config_path)
            model_path = '../models/bert/distilbert-transformers-pytorch/distilbert-base-uncased-pytorch_model.bin'
            config.output_hidden_states = True
            self.bert = DistilBertModel.from_pretrained(model_path, config=config)
        elif (model_type == 'bert'):
            config_path = '../models/bert/bert-base/bert-base-uncased-config.json'
            config = BertConfig.from_json_file(config_path)
            config.output_hidden_states = True
            model_path = '../models/bert/bert-base/bert-base-uncased-pytorch_model.bin'
            self.bert = BertModel.from_pretrained(model_path, config=config)
            
        self.dropout = nn.Dropout(p=0.2)
        self.high_dropout = nn.Dropout(p=0.5)   
        self.cls_token_head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(768 * 4, 768),
            nn.ReLU(inplace=True),
        )
        self.classifier = nn.Linear(768, 2)
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        
        if (self.model_type == 'roberta'):
            outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            hidden_layers = outputs[2]
        elif (self.model_type == 'distilbert'):
            outputs = self.bert(input_ids, attention_mask=attention_mask)
            hidden_layers = outputs[1]
        elif (self.model_type == 'bert'):     
            outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            hidden_layers = outputs[2]
        
        hidden_states_cls_embeddings = [x[:, 0] for x in hidden_layers[-4:]]
        x = torch.cat(hidden_states_cls_embeddings, dim=-1)
        cls_output = self.cls_token_head(x)
        logits = torch.mean(torch.stack([
            #Multi Sample Dropout takes place here
            self.classifier(self.high_dropout(cls_output))
            for _ in range(5)
        ], dim=0), dim=0)
        outputs = logits
        return outputs        

In [30]:
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        if len(np.unique(col_pred)) == 1:
            col_pred[np.random.randint(0, len(col_pred) - 1)] = col_pred.max() + 1
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

In [31]:
def train_and_predict(train_data, valid_data, test_data, epochs, batch_size, model_type, device, fold):
    
    dataloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, shuffle=False, batch_size=batch_size)
    test_dataloader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=batch_size)

    model = BertStyleModel(model_type).to(device)
    
    for n, _ in model.named_parameters():
        print(n)
 
    test_predictions = []
    valid_predictions = []

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if p.requires_grad and not any(nd in n for nd in no_decay) and "bert" in n],
            "weight_decay": 1e-2,
            "lr": 5e-5
        },
        {
            "params": [p for n, p in model.named_parameters() if  p.requires_grad and any(nd in n for nd in no_decay) and "bert" in n], 
            "weight_decay": 0.0,
            "lr": 5e-5
        },
        {
            "params": [p for n, p in model.named_parameters() if p.requires_grad and "bert" not in n],
            "weight_decay": 1e-2,
            "lr": 5e-4
            
        }
    ]
    optimizer = AdamW(optimizer_grouped_parameters)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(len(dataloader) * (epochs) * 0.05),
        num_training_steps=len(dataloader) * (epochs)
    )

    for epoch in range(epochs): 
        import time
        start = time.time()
        model.train()
        train_losses = []
        train_preds = []
        train_targets = []
        for input_ids, token_type_ids, attention_mask, targets in tqdm(dataloader, total=len(dataloader)):
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_mask = attention_mask.to(device)
            targets = targets.to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            
            train_preds.extend(outputs.detach().sigmoid().cpu().numpy())
            train_targets.extend(targets.detach().cpu().numpy())
            
            #loss = torch.nn.MultiLabelMarginLoss(outputs, targets)
            loss = F.binary_cross_entropy_with_logits(outputs, targets, reduction="none") 
            loss = (loss * LABEL_WEIGHTS[:loss.size(-1)]).mean()
            model.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_losses.append(loss.detach().cpu().item())
        
        model.eval()
        valid_losses = []
        valid_preds = []
        valid_targets = []
        with torch.no_grad():
            for input_ids, token_type_ids, attention_mask, targets in tqdm(valid_dataloader, total=len(valid_dataloader)):
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_mask = attention_mask.to(device)
                targets = targets.to(device)
                  
                outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                
                valid_preds.extend(outputs.sigmoid().cpu().numpy())
                valid_targets.extend(targets.cpu().numpy())
                
                #loss = torch.nn.MultiLabelMarginLoss(outputs, targets)
                loss = F.binary_cross_entropy_with_logits(outputs, targets, reduction="none")
                loss = (loss * LABEL_WEIGHTS[:loss.size(-1)]).mean()
                valid_losses.append(loss.detach().cpu().item())
            
            valid_predictions.append(np.stack(valid_preds))
            test_preds = []
            
            for input_ids, token_type_ids, attention_mask in tqdm(test_dataloader, total=len(test_dataloader)):
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_mask = attention_mask.to(device)
                
                outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                
                test_preds.extend(outputs.sigmoid().cpu().numpy())
            
            test_predictions.append(np.stack(test_preds))
    
        print("Epoch {}: Train Loss {}, Valid Loss {}".format(epoch + 1, np.mean(train_losses), np.mean(valid_losses)))
        print("\t Train Spearmanr {:.4f}, Valid Spearmanr (avg) {:.4f}, Valid Spearmanr (last) {:.4f}".format(
            compute_spearmanr(np.stack(train_targets), np.stack(train_preds)),
            compute_spearmanr(np.stack(valid_targets), sum(valid_predictions) / len(valid_predictions)),
            compute_spearmanr(np.stack(valid_targets), valid_predictions[-1])
        ))
        print("\t elapsed: {}s".format(time.time() - start))

    return valid_predictions, test_predictions

In [32]:
df_train, df_test = model_selection.train_test_split(data_df, shuffle=True, test_size=0.2)
df_train = df_train.reset_index(drop=True)
df_test= df_test.reset_index(drop=True)

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [34]:
LABEL_WEIGHTS = torch.tensor(1.0 / df_train[output_categories].std().values, dtype=torch.float32).to(device)
LABEL_WEIGHTS = LABEL_WEIGHTS / LABEL_WEIGHTS.sum() * 2
for name, weight in zip(output_categories, LABEL_WEIGHTS.cpu().numpy()):
    print(name, "\t", weight)

troll 	 0.72048426
controversiality 	 1.2795157


In [35]:
class Fold(object):
    def __init__(self, n_splits=5, shuffle=True, random_state=47):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def get_groupkfold(self, train, group_name):
        group = train[group_name]
        unique_group = group.unique()
        kf = KFold(
            n_splits=self.n_splits,
            shuffle=self.shuffle,
            random_state=self.random_state
        )
        
        folds_ids = []
        for trn_group_idx, val_group_idx in kf.split(unique_group):
            trn_group = unique_group[trn_group_idx]
            val_group = unique_group[val_group_idx]
            is_trn = group.isin(trn_group)
            is_val = group.isin(val_group)
            trn_idx = train[is_trn].index 
            val_idx = train[is_val].index
            folds_ids.append((trn_idx, val_idx))

        return folds_ids

In [36]:
gkf = Fold(n_splits=3, shuffle=True, random_state=47)
fold_ids = gkf.get_groupkfold(df_train, group_name="subreddit")

In [37]:
distilbert_tokenizer = DistilBertTokenizer(vocab_file = '../models/bert/distilbert-transformers-pytorch/bert-base-uncased-vocab.txt')
outputs = torch.tensor(compute_output_arrays(df_train), dtype=torch.float)
inputs = [torch.tensor(x, dtype=torch.long) for x in compute_input_arrays(df_train, distilbert_tokenizer)]
test_inputs = [torch.tensor(x, dtype=torch.long) for x in compute_input_arrays(df_test, distilbert_tokenizer)]

In [38]:
test_dataset = torch.utils.data.TensorDataset(*test_inputs)

In [39]:
print(outputs)

tensor([[0., 0.],
        [1., 0.],
        [0., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [0., 0.]])


In [40]:
histories = []

for fold, (train_idx, valid_idx) in enumerate(fold_ids):
    import gc
    gc.collect()
    
    train_inputs = [inputs[i][train_idx] for i in range(3)]
    train_outputs = outputs[train_idx]
    train_dataset = torch.utils.data.TensorDataset(*train_inputs, train_outputs)

    valid_inputs = [inputs[i][valid_idx] for i in range(3)]
    valid_outputs = outputs[valid_idx]
    valid_dataset = torch.utils.data.TensorDataset(*valid_inputs, valid_outputs)
   
    history = train_and_predict(train_data=train_dataset, valid_data=valid_dataset, test_data=test_dataset, 
        epochs=3, batch_size=8, model_type = 'distilbert', device = device, fold=fold)

    histories.append(history)

  0%|          | 0/733 [00:00<?, ?it/s]

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.transformer.layer.0.attention.q_lin.weight
bert.transformer.layer.0.attention.q_lin.bias
bert.transformer.layer.0.attention.k_lin.weight
bert.transformer.layer.0.attention.k_lin.bias
bert.transformer.layer.0.attention.v_lin.weight
bert.transformer.layer.0.attention.v_lin.bias
bert.transformer.layer.0.attention.out_lin.weight
bert.transformer.layer.0.attention.out_lin.bias
bert.transformer.layer.0.sa_layer_norm.weight
bert.transformer.layer.0.sa_layer_norm.bias
bert.transformer.layer.0.ffn.lin1.weight
bert.transformer.layer.0.ffn.lin1.bias
bert.transformer.layer.0.ffn.lin2.weight
bert.transformer.layer.0.ffn.lin2.bias
bert.transformer.layer.0.output_layer_norm.weight
bert.transformer.layer.0.output_layer_norm.bias
bert.transformer.layer.1.attention.q_lin.weight
bert.transformer.layer.1.attention.q_lin.bias
bert.transformer.layer.1.attenti

  0%|          | 0/733 [00:00<?, ?it/s]

Epoch 1: Train Loss 0.2397104445628893, Valid Loss 0.1606440605839989
	 Train Spearmanr 0.4958, Valid Spearmanr (avg) 0.5363, Valid Spearmanr (last) 0.5363
	 elapsed: 255.2367696762085s


  0%|          | 0/733 [00:00<?, ?it/s]

...

Epoch 1: Train Loss 0.20327538275134616, Valid Loss 0.23134682852536512
	 Train Spearmanr 0.5089, Valid Spearmanr (avg) 0.5313, Valid Spearmanr (last) 0.5313
	 elapsed: 287.4165608882904s


  0%|          | 0/921 [00:00<?, ?it/s]

Epoch 2: Train Loss 0.1538048578819602, Valid Loss 0.2040229191014905
	 Train Spearmanr 0.5662, Valid Spearmanr (avg) 0.5429, Valid Spearmanr (last) 0.5442
	 elapsed: 287.56513595581055s


100%|██████████| 336/336 [00:25<00:00, 13.04it/s]

Epoch 3: Train Loss 0.09910993997855001, Valid Loss 0.2615735959372002
	 Train Spearmanr 0.6359, Valid Spearmanr (avg) 0.5450, Valid Spearmanr (last) 0.5488
	 elapsed: 287.51194524765015s


In [41]:
val_preds_list = []
n_epochs = len(histories[0][0])

for epoch in range(n_epochs):
    val_preds_one_epoch = np.zeros([len(df_train), 2])    

    for fold, (train_idx, valid_idx) in enumerate(fold_ids):
        val_pred = histories[fold][0][epoch]
        val_preds_one_epoch[valid_idx, :] += val_pred

    val_preds_list.append(val_preds_one_epoch)

In [94]:
test_preds_list = []

#for epoch in range(n_epochs):
#    test_preds_one_epoch = 0

#    for fold in range(len(fold_ids)):
#        test_preds = histories[fold][1][epoch]
#        test_preds_one_epoch += test_preds

#    test_preds_one_epoch = test_preds_one_epoch / len(fold_ids)
#    test_preds_list.append(test_preds_one_epoch)


for epoch in range(n_epochs):
    test_preds_one_epoch = 0

    for fold in range(len(fold_ids)):
        test_preds = histories[fold][1][epoch]
        test_preds_one_epoch += test_preds

    test_preds_one_epoch = test_preds_one_epoch / len(fold_ids)
    test_preds_list.append(test_preds_one_epoch)

test_predictions_distil = np.zeros((n_epochs, len(df_test), len(output_categories)), dtype=np.float32)

for j, name in enumerate(output_categories):
    for epoch in range(n_epochs):
        col = "{}_{}".format(epoch, name)
        test_predictions_distil[epoch, :, j] = test_preds_list[epoch][:, j]

In [174]:
test_preds_distil = test_preds_list[n_epochs - 1][:, :]

In [175]:
test_preds_distil.shape

(2685, 2)

In [176]:
stats.describe(test_preds_distil[:,0])

DescribeResult(nobs=2685, minmax=(7.356166e-08, 1.0), mean=0.49959177, variance=0.24903183, skewness=0.002104994375258684, kurtosis=-1.9977449003094976)

In [177]:
stats.describe(test_preds_distil[:,1])

DescribeResult(nobs=2685, minmax=(3.5281304e-05, 0.8151059), mean=0.07728863, variance=0.0191027, skewness=2.6136560440063477, kurtosis=7.183695039175532)

In [178]:
y_test_troll = np.asarray(df_test['troll'])
y_test_contro = np.asarray(df_test['controversiality'])

In [179]:
print(classification_report(y_test_troll, np.around(test_preds_distil[:,0]).astype(np.int32)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1345
           1       1.00      1.00      1.00      1340

    accuracy                           1.00      2685
   macro avg       1.00      1.00      1.00      2685
weighted avg       1.00      1.00      1.00      2685



In [180]:
print(classification_report(y_test_contro, np.around(test_preds_distil[:,1]).astype(np.int32)))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      2443
           1       0.33      0.10      0.16       242

    accuracy                           0.90      2685
   macro avg       0.63      0.54      0.55      2685
weighted avg       0.86      0.90      0.88      2685



In [181]:
fpr_troll, tpr_troll, thresholds_troll = roc_curve(y_test_troll, np.around(test_preds_distil[:,0]).astype(np.int32))
fpr_contro, tpr_contro, thresholds_contro = roc_curve(y_test_contro, np.around(test_preds_distil[:,1]).astype(np.int32))

In [182]:
auc_troll = auc(fpr_troll, tpr_troll)
auc_contro = auc(fpr_contro, tpr_contro)
print(auc_troll, auc_contro)

0.9996282527881041 0.5414195728730764


In [183]:
x = y_test_troll - np.around(test_preds_distil[:,0]).astype(np.int32)
x.sum()

-1

In [53]:
ba_troll = balanced_accuracy_score(y_test_troll, np.around(test_preds_distil[:,0]).astype(np.int32))
ba_contro = balanced_accuracy_score(y_test_contro, np.around(test_preds_distil[:,1]).astype(np.int32))
print(ba_troll, ba_contro)

0.9996282527881041 0.5414195728730764


In [54]:
bert_tokenizer = BertTokenizer(vocab_file = '../models/bert/bert-base/bert-base-uncased-vocab.txt')
outputs = torch.tensor(compute_output_arrays(df_train), dtype=torch.float)
inputs = [torch.tensor(x, dtype=torch.long) for x in compute_input_arrays(df_train, bert_tokenizer)]
test_inputs = [torch.tensor(x, dtype=torch.long) for x in compute_input_arrays(df_test, bert_tokenizer)]
test_dataset = torch.utils.data.TensorDataset(*test_inputs)

In [55]:
bert_histories = []

for fold, (train_idx, valid_idx) in enumerate(fold_ids):
    import gc
    gc.collect()
    
    train_inputs = [inputs[i][train_idx] for i in range(3)]
    train_outputs = outputs[train_idx]
    train_dataset = torch.utils.data.TensorDataset(*train_inputs, train_outputs)

    valid_inputs = [inputs[i][valid_idx] for i in range(3)]
    valid_outputs = outputs[valid_idx]
    valid_dataset = torch.utils.data.TensorDataset(*valid_inputs, valid_outputs)
   
    history = train_and_predict(train_data=train_dataset, valid_data=valid_dataset, test_data=test_dataset, 
        epochs=3, batch_size=8, model_type = 'bert', device = device, fold=fold)

    bert_histories.append(history)

  0%|          | 0/733 [00:00<?, ?it/s]

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

  0%|          | 0/733 [00:00<?, ?it/s]

Epoch 1: Train Loss 0.25398403245544043, Valid Loss 0.1635389211725016
	 Train Spearmanr 0.4851, Valid Spearmanr (avg) 0.5402, Valid Spearmanr (last) 0.5402
	 elapsed: 495.59016609191895s


  0%|          | 0/733 [00:00<?, ?it/s]

...

Epoch 1: Train Loss 0.20821738861507283, Valid Loss 0.24535326843737001
	 Train Spearmanr 0.5133, Valid Spearmanr (avg) 0.5052, Valid Spearmanr (last) 0.5052
	 elapsed: 555.4103755950928s


  0%|          | 0/921 [00:00<?, ?it/s]

Epoch 2: Train Loss 0.15871176949320978, Valid Loss 0.2078020681861029
	 Train Spearmanr 0.5615, Valid Spearmanr (avg) 0.5555, Valid Spearmanr (last) 0.5571
	 elapsed: 555.9400916099548s


100%|██████████| 336/336 [00:51<00:00,  6.58it/s]

Epoch 3: Train Loss 0.10767920500518963, Valid Loss 0.24470226979434878
	 Train Spearmanr 0.6274, Valid Spearmanr (avg) 0.5530, Valid Spearmanr (last) 0.5338
	 elapsed: 556.6648147106171s


In [171]:
n_epochs = len(bert_histories[0][0])
test_bert_preds_list = []
test_predictions_bert = np.zeros((n_epochs, len(df_test), len(output_categories)), dtype=np.float32)

for epoch in range(n_epochs):
    test_preds_one_epoch_bert = []

    for fold in range(len(fold_ids)):
        test_preds = bert_histories[fold][1][epoch]
        test_preds_one_epoch_bert.append(test_preds)
   
    test_preds_one_epoch_bert = np.array([np.array([rankdata(c) for c in p.T]).T 
                                          for p in test_preds_one_epoch_bert]).mean(axis=0)
    max_epoch_rank_value = (test_preds_one_epoch_bert).max() + 1
    test_preds_one_epoch_bert = test_preds_one_epoch_bert/max_epoch_rank_value
    test_bert_preds_list.append(test_preds_one_epoch_bert)


for j, name in enumerate(output_categories):
    for epoch in range(n_epochs):
        col = "{}_{}".format(epoch, name)
        test_predictions_bert[epoch, :, j] = test_bert_preds_list[epoch][:, j]

In [172]:
test_predictions_bert.shape

(3, 2685, 2)

In [173]:
test_preds_bert = test_bert_preds_list[n_epochs - 1][:, :]

In [170]:
stats.describe(test_preds_bert[:,0])

DescribeResult(nobs=2685, minmax=(0.0014903129657228018, 0.9909960258320915), mean=0.5003725782414307, variance=0.07692651083590676, skewness=-0.03004033354239729, kurtosis=-1.3721339349705053)

In [158]:
stats.describe(test_preds_bert[:,1])

DescribeResult(nobs=2685, minmax=(0.001366120218579235, 0.9996274217585693), mean=0.5003725782414307, variance=0.06407081454996681, skewness=0.08559423095304955, kurtosis=-1.0506793559002334)

In [159]:
print(classification_report(y_test_troll, np.around(test_preds_bert[:,0]).astype(np.int32)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1345
           1       1.00      1.00      1.00      1340

    accuracy                           1.00      2685
   macro avg       1.00      1.00      1.00      2685
weighted avg       1.00      1.00      1.00      2685



In [160]:
print(classification_report(y_test_contro, np.around(test_preds_bert[:,1]).astype(np.int32)))

              precision    recall  f1-score   support

           0       0.97      0.55      0.70      2443
           1       0.16      0.84      0.26       242

    accuracy                           0.58      2685
   macro avg       0.56      0.70      0.48      2685
weighted avg       0.90      0.58      0.66      2685



In [161]:
fpr_troll, tpr_troll, thresholds_troll = roc_curve(y_test_troll, np.around(test_preds_bert[:,0]).astype(np.int32))
fpr_contro, tpr_contro, thresholds_contro = roc_curve(y_test_contro, np.around(test_preds_bert[:,1]).astype(np.int32))

In [162]:
auc_troll = auc(fpr_troll, tpr_troll)
auc_contro = auc(fpr_contro, tpr_contro)
print(auc_troll, auc_contro)

0.9973963824002664 0.6963545701498293


In [184]:
x = y_test_troll - np.around(test_preds_bert[:,0]).astype(np.int32)
x.sum()

-5

In [164]:
ba_troll = balanced_accuracy_score(y_test_troll, np.around(test_preds_bert[:,0]).astype(np.int32))
ba_contro = balanced_accuracy_score(y_test_contro, np.around(test_preds_bert[:,1]).astype(np.int32))
print(ba_troll, ba_contro)

0.9973963824002663 0.6963545701498293


In [93]:
roberta_tokenizer_merges_file = '../models/bert/roberta-transformers-pytorch/roberta-base/roberta-base-merges.txt'
roberta_tokenizer = RobertaTokenizer(vocab_file = '../models/bert/roberta-transformers-pytorch/roberta-base/roberta-base-vocab.json', merges_file=roberta_tokenizer_merges_file)
outputs = torch.tensor(compute_output_arrays(df_train), dtype=torch.float)
inputs = [torch.tensor(x, dtype=torch.long) for x in compute_input_arrays(df_train, roberta_tokenizer)]
test_inputs = [torch.tensor(x, dtype=torch.long) for x in compute_input_arrays(df_test, roberta_tokenizer)]